In [1]:
import os
import joblib
import keras.models

os.chdir('/content/drive/Othercomputers/AKATSUKI-PC/PycharmProjects/chatopotamus')

In [7]:
import json

def convert_to_IOB(data, lookup):
	converted_data = []

	for instance in data['instances']:
		sentence = instance['sentence']
		words = sentence.split()

		# Initialize tags as 'O' for Outside
		tags = ['O'] * len(words)

		# Function to tag words based on entities
		def tag_words(entity_type, entity_values):
			for entity_value in entity_values:
				start = sentence.find(entity_value)
				if start != -1:
					# Calculate end position of the entity
					end = start + len(entity_value)
					start_word_count = len(sentence[:start].split())
					end_word_count = len(sentence[:end].split())
					for i in range(start_word_count, end_word_count):
						if i == start_word_count:
							tags[i] = 'B-' + entity_type
						else:
							tags[i] = 'I-' + entity_type

		# Tagging each entity type
		for entity_type, entity_values in lookup['entities'].items():
			tag_words(entity_type, entity_values)

		converted_data.append({"sentence": sentence, "tags": tags})

	return converted_data

# Example usage
with open('resources/bert/data/ner_dataset.json', 'r') as file:
	ner_dataset_json = json.load(file)

with open('resources/bert/data/ner_lookup.json', 'r') as file:
	ner_lookup = json.load(file)

In [22]:
iob_dataset = convert_to_IOB(ner_dataset_json, ner_lookup)

In [ ]:
from transformers import BertTokenizer, TFBertForTokenClassification
import tensorflow as tf

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

num_labels = len(ner_lookup['entities'])*2 + 1

# Load pre-trained model (weights)
model = TFBertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

**For Reloading/Saving**

In [ ]:
# Save important variables
joblib.dump(iob_dataset, 'resources/bert/data/iob_dataset.pkl')
joblib.dump(tokenizer, 'resources/bert/pretrained/tokenizer.pkl')
model.save('resources/bert/pretrained/bert-base-uncased.keras')

In [ ]:
# Load variables
tokenizer = joblib.load('resources/bert/pretrained/tokenizer.pkl')
model = keras.models.load_model('resources/bert/pretrained/bert-base-uncased.keras')
iob_dataset = joblib.load('resources/bert/data/iob_dataset.pkl')

In [64]:
iob_dataset[0]

{'sentence': 'i ordered a medium double-double and a boston cream donut from tim hortons',
 'tags': ['O',
  'O',
  'O',
  'B-beverage_size',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-food',
  'O',
  'O',
  'O']}